# Section 3.3: Factor Analysis

## Section 3.3.1.0: Package initialisations, environment configuration and function definitions

Import relevant packages:

In [8]:
import tensorflow as tf
import numpy as np

import time
import datetime
import os

# TensorFlow embedding API library
from tensorflow.contrib.tensorboard.plugins import projector

# Non-interactive plotting
import matplotlib.pyplot as plt

# Interactive plotting
from plotly import tools
import plotly.plotly as py
import plotly.graph_objs as go
import plotly.offline as pyo
from plotly.offline import download_plotlyjs

Configure environment:

In [9]:
%config InlineBackend.figure_format = 'retina'
np.set_printoptions(precision=3)

# Global Variables
LOG_DIR = './Logs'

# Activate Plotly Offline for Jupyter
pyo.init_notebook_mode(connected=True)

# Define global variable SEED
SEED = 521

Load data2D.npy and data100D.npy into memory:

In [119]:
"""
tinymnist.npz consists of images of '3's and '5's with dimensions (8 x 8)

train_data: 700 data points
valid_data: 100 data points
test_data: 400 data points
"""
with np.load ("./Data/tinymnist.npz") as data :
    # Generate _data
    train_data, train_target = data ["x"], data["y"]
    valid_data, valid_target = data ["x_valid"], data ["y_valid"]
    test_data, test_target = data ["x_test"], data ["y_test"]

### Load results (Optional; when working resuming work session)

In [12]:
# results_3_1_2 = np.load('./Results/FA/3_1_2.npy')
# results_3_1_3 = np.load('./Results/FA/3_1_3.npy')
# results_3_1_3_x5000 = np.load('./Results/FA/3_1_3_x5000.npy')

## Create Factor Analysis (FA) TensorFlow graph:

### Loss function:

$$ \min_{W, \Psi} - \log P(\mathbf{X}) = - \sum_{n=1}^B \log \mathcal{N}(\mathbf{x}; \mathbf{\mu}, \mathbf{\Psi} + \mathbf{WW}^T) $$

In [120]:
'''
Builds TensorFlow graph for FA

Input:
    K: number of latent variables
    D: dataset dimension
    device: CPU or GPU to perform computation-heavy ops
Internal variables:
    X: input data matrix (N x D)
    mu: mean of each input (D x 1)
    psi_vector: variance of x_n given s_n for each dimension (D x 1)
    W: latent_matrix that projects s_n from K-dimensions to D-dimensions (K x D)
    Sigma: Marginal covariance matrix (D x D)
'''
def build_FA(K, D, device='cpu'):
    '''
    Helper function to add histogram tag to variables
    Input:
        var: variable to be tagged with histogram summary
    '''
    def _add_histogram(vars_):
        for var in vars_:
            tf.summary.histogram(var.op.name, var)
    
    #######################
    ##  Function begins  ##
    #######################
    
    # Define computation device
    try:
        assert device == 'cpu' or device == 'gpu'
    except AssertionError:
        print 'Invalid device chosen. Please use \'cpu\' or \'gpu\''
        quit()
    device = '/' + device + ':0'
    
    SEED = 522
    
    # Fix TF graph seed
    tf.set_random_seed(SEED)
    
    with tf.device('/cpu:0'):
        # Define placeholder
        with tf.name_scope('placeholder'):
            X = tf.placeholder(tf.float32, shape=[None, D], name='inputs')
            
        # Define parameters
        with tf.variable_scope('generated_parameters'):
            phi = tf.get_variable('latent_for_psi', shape=[D, 1], \
                                initializer=tf.truncated_normal_initializer(seed=SEED))
            W = tf.get_variable('latent_matrix_W', shape=[D, K], \
                                       initializer=tf.truncated_normal_initializer(seed=SEED))
            mu = tf.get_variable('mean', shape=[D, 1], initializer=tf.truncated_normal_initializer(seed=SEED))
        
    with tf.device(device):
        # Calculate gaussian parameters
        with tf.name_scope('gaussian_parameters'):
            psi_vector = tf.exp(phi, name='variance_vector_psi')
            Psi = tf.multiply(tf.eye(tf.shape(psi_vector)[0]), psi_vector, name='Psi_matrix')
            Sigma = tf.add(Psi, tf.matmul(W, tf.transpose(W)), name='Sigma')
            
        # Calculate projection matrix to infer latent variable s
        with tf.name_scope('projection_matrix'):
            Sigma_s_posterior = tf.matrix_inverse(tf.eye(K) \
                                       + tf.matmul(tf.transpose(W), \
                                                   tf.matmul(tf.matrix_inverse(Psi), W)))
            W_proj = tf.matmul(Sigma_s_posterior, tf.matmul(tf.transpose(W), tf.matrix_inverse(Psi)))
            
        # Calculate loss function
        with tf.name_scope('marginal_log_likelihood'):
            with tf.name_scope('Mahalanobis_dist'):
                # Expand variables for tensor multiplication
                X_expanded = tf.expand_dims(X, axis=2)
                mu_expanded = tf.expand_dims(mu, axis=0)

                # Calculate mahalanobis distance
                Sigma_inv_tiled = tf.tile(tf.expand_dims(tf.matrix_inverse(Sigma), axis=0), \
                                          multiples=[tf.shape(X)[0],1,1], \
                                          name='Sigma_inv_tiled')
                
                dist = tf.reduce_sum(- 1. / 2 * \
                                   tf.matmul(tf.transpose(X_expanded - mu_expanded, perm=[0,2,1]), \
                                             tf.matmul(Sigma_inv_tiled, \
                                                       X_expanded - mu_expanded)), \
                                   name='Mahalanobis_dist')
        
            # Calculate log gaussian constant
            with tf.name_scope('log_gauss_const'):
                log_gauss_const = tf.negative(tf.cast(tf.shape(X)[0], tf.float32) * tf.cast(D, tf.float32)\
                                              * tf.log(2. * np.pi) / 2,\
                                              name='log_gauss_const')
            
            # Calculate log_determinant
            with tf.name_scope('log_det'):
                log_det = tf.negative(tf.cast(tf.shape(X)[0], tf.float32) \
                                      * tf.reduce_sum(tf.log(tf.diag_part(tf.cholesky(Sigma)))),\
                                      name='log_det')
            
            # Calculate loss function
            loss = tf.negative(tf.add_n([dist, log_gauss_const, log_det]), name='loss')
            tf.summary.scalar('loss', loss)
        
        # Define optimizer
        with tf.name_scope('Adam_optimizer'):
            optimizer = tf.train.AdamOptimizer(learning_rate=0.01, \
                                               beta1=0.9, beta2=0.99, epsilon=1e-5).minimize(loss)
        
    with tf.device('/cpu:0'):
        # Add histogram summaries for variables of interest
        _add_histogram([psi_vector, W, Sigma])
        
        # Merge all summaries
        merged = tf.summary.merge_all()
        
    return X, mu, psi_vector, W, Sigma, W_proj, loss, optimizer, merged

### Define training function:

In [121]:
'''
Runs FA training algorithm. Tensorboard embedding enabled
'''
def run_FA(K_list, QUES_DIR, D=64, device='cpu'):    
    '''
    Embed data for visualization purposes
    '''
    def embed_data(D, train_writer):
        # Define input data
        if D == 64:
            input_data = train_data
            input_data_name = 'tinymnist_training'
        elif D == 3:
            input_data = toy_data
            input_data_name = 'toy_data'
        
        # Create variable to embed
        data_to_embed = tf.Variable(input_data, name=input_data_name, trainable=False, collections=[])

        # Define projector configurations
        config = projector.ProjectorConfig()
        
        # Add embedding
        embedding = config.embeddings.add()
        
        # Connect tf.Variable to embedding
        embedding.tensor_name = data_to_embed.name

        # Evaluate tf.Variable
        sess.run(data_to_embed.initializer)
        
        # Create save checkpoint
        saver = tf.train.Saver([data_to_embed])
        saver.save(sess, SUMMARY_DIR + sub_dir + '/train/model.ckpt', MAX_ITER)

        # Write projector_config.pbtxt in LOG_DIR
        projector.visualize_embeddings(train_writer, config)
    
    #######################
    ##  Function begins  ##
    #######################    
    
    # Check compatibility of dataset
    try:
        assert D == 64 or D == 3
    except AssertionError:
        print 'Incompatible dataset dimension, D. Please use 64 or 3\
            for tinymnist or toy dataset respectively'
        quit()
    
    # Define locally global function
    MAX_ITER = 1200 if D == 64 else 5000
    CURR_TIME = '{:%b%d %H_%M_%S}'.format(datetime.datetime.now())
    SUMMARY_DIR = CURRENT_DIR + LOG_DIR + '/FA/' + QUES_DIR + '/' + CURR_TIME
    
    # Create list to store run results
    results = []
    
    for K in K_list:
        # Clear any pre-defined graph
        tf.reset_default_graph()
        
        # Build TensorFlow graph
        X, mu, psi_vector, W, Sigma, W_proj, loss, optimizer, merged = build_FA(K, D, device)
        
        # Select appropriate input_data
        if D == 64:
            input_data = train_data
        elif D == 3:
            input_data = toy_data

        # Create arrays to log losses, psi and W
        train_loss = np.array([])[:, np.newaxis]
        if D == 64:
            valid_loss = np.array([])[:, np.newaxis]
            test_loss = np.array([])[:, np.newaxis]
        
        mean = np.array([])[:, np.newaxis].reshape(0, D)
        psi = np.array([])[:, np.newaxis].reshape(0, D)
        Ws = np.array([])[:, np.newaxis, np.newaxis].reshape(0, D, K)
        Wproj = np.array([])[:, np.newaxis, np.newaxis].reshape(0, K, D)
        
        # Begin session
        with tf.Session(config=tf.ConfigProto(allow_soft_placement=True, log_device_placement=False)) as sess:
            # Log start time
            start_time = time.time()

            # Create sub-directory title
            sub_dir = '/K={},D={}'.format(K, D)
            
            # Create summary writers
            train_writer = tf.summary.FileWriter(SUMMARY_DIR + sub_dir + '/train', graph=sess.graph)
            if D == 64:
                valid_writer = tf.summary.FileWriter(SUMMARY_DIR + sub_dir + '/valid')
                test_writer = tf.summary.FileWriter(SUMMARY_DIR + sub_dir + '/test')
                
            # Initialise all TensorFlow variables
            tf.global_variables_initializer().run()
            
            # Define iterator
            curr_iter = 0
            
            # Calculate training (and validation) loss, 
            # cluster centres and responsibility indices before any training
            err, summaries, curr_mu, curr_psi, curr_W, curr_W_proj = \
                sess.run([loss, merged, mu, psi_vector, W, W_proj], feed_dict={X: input_data})
            train_loss = np.append(train_loss, err)
            train_writer.add_summary(summaries, curr_iter)
            
            # Log psi and W
            mean = np.append(mean, np.transpose(curr_mu), axis=0)
            psi = np.append(psi, np.transpose(curr_psi), axis=0)
            Ws = np.append(Ws, curr_W[np.newaxis, :, :], axis=0)
            Wproj = np.append(Wproj, curr_W_proj[np.newaxis, :, :], axis=0)
            
            if D == 64:
                # Log validation loss
                err, summaries  = sess.run([loss, merged], feed_dict={X:valid_data})
                valid_loss = np.append(valid_loss, err)
                valid_writer.add_summary(summaries, curr_iter)

                # Log test loss
                err, summaries  = sess.run([loss, merged], feed_dict={X:test_data})
                test_loss = np.append(test_loss, err)
                test_writer.add_summary(summaries, curr_iter)
            
            # Begin training
            while curr_iter < MAX_ITER:                
                # Train graph
                _, summaries, err = sess.run([optimizer, merged, loss], feed_dict={X:input_data})
                
                # Add training loss
                train_loss = np.append(train_loss, err)
                train_writer.add_summary(summaries, curr_iter + 1)

                if D == 64:
                    # Log validation loss
                    summaries, err = sess.run([merged, loss], feed_dict={X:valid_data})
                    valid_loss = np.append(valid_loss, err)
                    valid_writer.add_summary(summaries, curr_iter)

                    # Log test loss
                    err, summaries  = sess.run([loss, merged], feed_dict={X:test_data})
                    test_loss = np.append(test_loss, err)
                    test_writer.add_summary(summaries, curr_iter)
                
                # Log responsibility indices and cluster centres every 10% of maximum iteration
                if ((float(curr_iter) + 1) * 100 / MAX_ITER) % 10 == 0:
                    curr_mu, curr_psi, curr_W, curr_W_proj = sess.run([mu, psi_vector, W, W_proj], feed_dict={X:input_data})
                    
                    mean = np.append(mean, np.transpose(curr_mu), axis=0)
                    psi = np.append(psi, np.transpose(curr_psi), axis=0)
                    Ws = np.append(Ws, curr_W[np.newaxis, :, :], axis=0)
                    Wproj = np.append(Wproj, curr_W_proj[np.newaxis, :, :], axis=0)
                
                # Post training progress to user, every 100 iterations
                if curr_iter % 100 == 99:
                    print 'iter: {:3d}, train_loss: {:5.0f}'.format(curr_iter + 1, train_loss[-1])
                
                curr_iter += 1
            
            # End of while loop
            print 'Max iteration reached'
            
            # Embed data
            embed_data(D, train_writer)
            
            # Close writers
            train_writer.close()
            if D == 64:
                valid_writer.close()
                test_writer.close()

            if D == 64:
                results.append(
                    {
                        'K': K,
                        'train_loss': train_loss,
                        'valid_loss': valid_loss,
                        'test_loss': test_loss,
                        'psi': psi,
                        'mean': mean,
                        'W': Ws,
                        'W_proj': Wproj,
                        'time_of_run': '{:%b%d %H_%M_%S}'.format(datetime.datetime.now())
                    }
                )
            elif D == 3:
                results.append(
                    {
                        'K': K,
                        'train_loss': train_loss,
                        'psi': psi,
                        'mean': mean,
                        'W': Ws,
                        'W_proj': Wproj,
                        'time_of_run': '{:%b%d %H_%M_%S}'.format(datetime.datetime.now())
                    }
                )
            
            # TODO calculate convergence
            print 'K: {:3d}, duration: {:3.1f}s\n'\
                .format(K, time.time() - start_time)
                                                                              
    print 'RUN COMPLETED'
    return results

## Section 3.3.1.2: FA on $\textit{tinymnist.npz}$ with validation $(K = 4)$

In [122]:
results_3_1_2 = run_FA(K_list=[4], QUES_DIR='/Q3.1.2', device='gpu')

iter: 100, train_loss: 31080
iter: 200, train_loss: 15126
iter: 300, train_loss:  2180
iter: 400, train_loss: -4400
iter: 500, train_loss: -6865
iter: 600, train_loss: -8385
iter: 700, train_loss: -8445
iter: 800, train_loss: -8453
iter: 900, train_loss: -8453
iter: 1000, train_loss: -8453
iter: 1100, train_loss: -8453
iter: 1200, train_loss: -8453
Max iteration reached
K:   4, duration: 54.8s

RUN COMPLETED


### Save results

In [123]:
np.save('./Results/FA/3_1_2.npy', results_3_1_2)

### Visualise weights

In [125]:
'''
Creates 2x2 subplot of weights
Each subplot consists of an 8x8 heatmap for the weight of each latent variable
Input:
    weights: final trained weights (D x K)
'''
def visualise_weights(weights):
    # Define colour list as per Plotly's default colour list
    colour_list = np.array(['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b'])
    
    # Define empty figure
    figure = tools.make_subplots(rows=2, cols=2, subplot_titles=('Weight 1', 'Weight 2', 'Weight 3', 'Weight 4'))
    
    # Define subplot traces
    for i, weight in enumerate(np.transpose(weights)):
        trace = go.Heatmap(
            z = np.reshape(weight, (8,8)), # TODO Reverse order
            colorscale = [[0, '#000000'], [1, '#FFFFFF']],
            showscale = False
        )
        figure.append_trace(trace, i / 2 + 1, i % 2 + 1)
        
        figure['layout']['xaxis{}'.format(i + 1)].update(showticklabels = False, ticks = '')
        figure['layout']['yaxis{}'.format(i + 1)].update(showticklabels = False, ticks = '', autorange='reversed')
        
    figure['layout'].update(
        height = 900,
        width = 800,
        showlegend = False,
        title = 'Weights Visualisation of Latent Matrix, W'
    )
    
    py.iplot(figure, filename='/ECE521: A3/Q3: Factor Analysis/Q1.2_weights_viz', sharing='private')
    return pyo.iplot(figure)

visualise_weights(results_3_1_2[0]['W'][-1])

[ 0.041  0.112  0.198  0.301  0.404  0.467  0.472  0.303  0.072  0.14
  0.228  0.288  0.355  0.356  0.364  0.254  0.059  0.152  0.196  0.233
  0.265  0.26   0.245  0.082  0.047  0.176  0.321  0.44   0.486  0.439
  0.258  0.042  0.032  0.08   0.142  0.19   0.224  0.315  0.382  0.142
  0.12   0.058  0.034  0.062  0.103  0.256  0.38   0.2    0.32   0.32
  0.234  0.241  0.315  0.402  0.327  0.109  0.182  0.448  0.552  0.537
  0.42   0.246  0.095  0.014]
This is the format of your plot grid:
[ (1,1) x1,y1 ]  [ (1,2) x2,y2 ]
[ (2,1) x3,y3 ]  [ (2,2) x4,y4 ]



# Section 3.3.1.3: PCA vs FA

### Generate toy dataset of 200, 3D points using the following relations:

$$ x_1 = s_1 \\ x_2 = s_1 + 0.001 s_2 \\ x_3 = 10 s_3 $$

In [44]:
# Reset random seed
np.random.seed(SEED)

# Define variable to store toy data
toy_data = np.array([])[np.newaxis, :].reshape(0, 3)

for i in range(200):
    # Sample s from normal distribution
    s = np.random.randn(3,1)
    
    # Define conversion matrix from s to x
    A = np.array([[1, 0, 0], 
                  [1, 0.001, 0], 
                  [0, 0, 10]])
    
    toy_data = np.append(toy_data, np.transpose(np.matmul(A, s)), axis=0)

## Train using PCA

In [46]:
'''
Finds the largest principal component by finding the normalised eigenvector corresponding
to the largest eigenvalue of the data covariance matrix
'''
def largest_component_PCA(data):
    # Obtain covariance matrix
    sigma = np.cov(np.transpose(data))

    # Calculate eigenvalues and eigenvectors
    e_value, e_vector = np.linalg.eig(sigma)
    
    # Return the largest principle component
    PC = e_vector[np.argmax(e_value)][:, np.newaxis]
    
    # Save data
    np.save('./Results/FA/3_1_3_PCA_x{}.npy'.format(data.shape[0]), PC)
    
    return PC

print 'Principle component: \n{}'.format(largest_component_PCA(toy_data))

Principle component: 
[[  1.212e-06]
 [  1.373e-03]
 [  1.000e+00]]


## Train using FA

In [19]:
results_3_1_3 = run_FA(K_list=[1], D=3, QUES_DIR='Q3.1.3', device='cpu')

iter: 100, train_loss:  4166
iter: 200, train_loss:  2799
iter: 300, train_loss:  2317
iter: 400, train_loss:  2018
iter: 500, train_loss:  1816
iter: 600, train_loss:  1673
iter: 700, train_loss:  1570
iter: 800, train_loss:  1496
iter: 900, train_loss:  1441
iter: 1000, train_loss:  1401
iter: 1100, train_loss:  1371
iter: 1200, train_loss:  1349
iter: 1300, train_loss:  1332
iter: 1400, train_loss:  1322
iter: 1500, train_loss:  1320
iter: 1600, train_loss:  1320
iter: 1700, train_loss:  1150
iter: 1800, train_loss:  1018
iter: 1900, train_loss:   904
iter: 2000, train_loss:   795
iter: 2100, train_loss:   690
iter: 2200, train_loss:   586
iter: 2300, train_loss:   484
iter: 2400, train_loss:   384
iter: 2500, train_loss:   286
iter: 2600, train_loss:   218
iter: 2700, train_loss:   158
iter: 2800, train_loss:   125
iter: 2900, train_loss:   228
iter: 3000, train_loss:   129
iter: 3100, train_loss:   172
iter: 3200, train_loss:   353
iter: 3300, train_loss:   167
iter: 3400, train_l

In [22]:
results_3_1_3_x5000 = run_FA(K_list=[1], D=3, QUES_DIR='Q3.1.3', device='cpu')

iter: 100, train_loss: 100993
iter: 200, train_loss: 68378
iter: 300, train_loss: 56776
iter: 400, train_loss: 49570
iter: 500, train_loss: 44687
iter: 600, train_loss: 41249
iter: 700, train_loss: 38779
iter: 800, train_loss: 36983
iter: 900, train_loss: 35668
iter: 1000, train_loss: 34703
iter: 1100, train_loss: 33991
iter: 1200, train_loss: 33455
iter: 1300, train_loss: 33057
iter: 1400, train_loss: 32860
iter: 1500, train_loss: 32838
iter: 1600, train_loss: 32837
iter: 1700, train_loss: 32837
iter: 1800, train_loss: 32837
iter: 1900, train_loss: 32837
iter: 2000, train_loss: 32837
iter: 2100, train_loss: 32837
iter: 2200, train_loss: 28494
iter: 2300, train_loss: 25172
iter: 2400, train_loss: 22332
iter: 2500, train_loss: 19650
iter: 2600, train_loss: 17046
iter: 2700, train_loss: 14492
iter: 2800, train_loss: 11973
iter: 2900, train_loss:  9503
iter: 3000, train_loss:  7037
iter: 3100, train_loss:  5225
iter: 3200, train_loss:  4304
iter: 3300, train_loss:  2752
iter: 3400, train_

### Save FA results

In [80]:
np.save('./Results/FA/3_1_3.npy', results_3_1_3)
np.save('./Results/FA/3_1_3_x5000.npy', results_3_1_3_x5000)

In [24]:
print results_3_1_3[0]['W'][-1]
print results_3_1_3_x5000[0]['W'][-1]

[[ 1.06 ]
 [ 1.065]
 [-0.1  ]]
[[ 1.047]
 [ 1.043]
 [-0.032]]


In [23]:
print results_3_1_3[0]['W_proj'][-1]
print results_3_1_3_x5000[0]['W_proj'][-1]

[[  4.798e-01   4.616e-01  -3.145e-09]]
[[  4.771e-01   4.801e-01  -1.198e-09]]
